### Validation of Ghazal-Level Semantic Axes (YaarAI)

This notebook validates the **ghazal_axis** annotations generated for the YaarAI project.

The goal is to ensure that:
- each ghazal has a valid abstract semantic axis,
- axes are concise noun phrases (not prose),
- no verbal or directive language leaks into the annotations,
- axes apply coherently across all bayts of a ghazal.

This notebook is intended for **inspection and validation only**.
No annotation or regeneration is performed here.


In [1]:
import json
import random
import re
from pathlib import Path
from collections import Counter, defaultdict

### Load ghazal axis annotations

We load the ghazal-level semantic axis annotations from disk and verify
that the expected number of entries is present.

In [2]:
PATH = Path("../data/annotations/ghazal_axis_v1.jsonl")

rows = []
with PATH.open("r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

len(rows) # there are 495 ghazals in Divan-e-Hafez

495

### Schema sanity check

Each record must:
- contain an integer `poem_id`
- contain a non-empty string `ghazal_axis`


In [3]:
for r in rows:
    assert isinstance(r["poem_id"], int)
    assert isinstance(r["ghazal_axis"], str)
    assert r["ghazal_axis"].strip()

### Axis length statistics

We inspect the length (in words) of ghazal axes to ensure they remain
short abstract noun phrases rather than explanatory prose.


In [4]:
lengths = [len(r["ghazal_axis"].split()) for r in rows]

min(lengths), max(lengths), round(sum(lengths) / len(lengths), 2)

(2, 10, 4.05)

### Inspection of long axes

Axes with unusually large word counts are inspected manually to ensure
they are still semantically justified and not overly verbose.


In [5]:
long_axes = [r for r in rows if len(r["ghazal_axis"].split()) >= 8]
len(long_axes)

7

In [6]:
for r in long_axes:
    print(len(r["ghazal_axis"].split()), "→", r["ghazal_axis"])

8 → تغییر و دگرگونی در پیوندهای انسانی و معنوی
8 → جستجوی حقیقت و معرفت در پیوند با عشق
9 → پیوند مراتب وجودی و تمایز میان ظاهر و باطن
10 → پیوند رمز و نشانه‌های کیهانی با مفاهیم وفا و معرفت
8 → جستجوی کمال و معنا در پیوند با معشوق
8 → جستجوی معنای پایدار در پیوند انسانی و الهی
8 → جستجوی حقیقت در پیوند با معشوق و می


### Verb-surface diagnostic (heuristic)

As a diagnostic step, we scan for verb-like surface forms.
This is not an error detector, but a sanity check against accidental
prose leakage.

In [7]:
verb_like = re.compile(r"(می‌|می |کرد|شد|است|بود)")

bad = [r for r in rows if verb_like.search(r["ghazal_axis"])]
print(len(bad))
bad

9


[{'poem_id': 83,
  'ghazal_axis': 'گذرا بودن وقایع و احوال',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 96,
  'ghazal_axis': 'درخواست یاری در مواجهه با رنج\u200cهای بی\u200cپایان',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 168,
  'ghazal_axis': 'ناکامی در تحقق آرمان',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 174,
  'ghazal_axis': 'بازگشت و نو شدن',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 194,
  'ghazal_axis': 'تعامل ناپایدار میان جذابیت و دست\u200cنیافتنی بودن',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 208,
  'ghazal_axis': 'نبود ارزش\u200cهای بنیادین در زندگی',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 358,
  'ghazal_axis': 'نبود امکان وصول حقیقت',
  'annotation_meta': {'model': 'gpt-4.1', 'prompt_version': 'v1.0'}},
 {'poem_id': 405,
  'gh

### Axis frequency distribution

We inspect the most frequent axes to understand the high-level
semantic structure of the dataset and detect possible degeneracy.

In [8]:
axes = [r["ghazal_axis"] for r in rows]
Counter(axes).most_common(20)

[('جستجوی حقیقت پنهان', 12),
 ('ناپایداری جهان', 4),
 ('تمایز حقیقت و ظاهر', 4),
 ('تقابل ظاهر و باطن', 4),
 ('جستجوی وصال', 4),
 ('جستجوی حقیقت مطلق', 3),
 ('کشمکش درونی', 2),
 ('پایداری پیوند معنوی', 2),
 ('مرکزیت معشوق', 2),
 ('پیوند ناپیدای دل و معشوق', 2),
 ('جستجوی پیوند معنوی', 2),
 ('پیوند ناپیدای عاشق و معشوق', 2),
 ('جستجوی کمال', 2),
 ('جدایی و پیوستگی', 2),
 ('ناپایداری دستاوردها', 2),
 ('پیچیدگی روابط انسانی', 2),
 ('ترک تعلقات دنیوی', 2),
 ('جدایی و بی\u200cاعتنایی', 2),
 ('جستجوی حقیقت درونی', 2),
 ('ناپایداری فرصت\u200cها', 2)]

### Deep inspection of a frequent axis

We select a frequent axis and inspect all bayts of its ghazals to ensure
the axis applies coherently across the poem.


In [9]:
target_axis = "جستجوی حقیقت پنهان"

matching = [
    r for r in rows
    if r["ghazal_axis"] == target_axis
]

len(matching)

12

### Load raw ghazal text

To contextualize axes, we load the original bayts for each ghazal.

In [10]:
RAW_PATH = Path("../data/raw/ghazals_with_insight.jsonl")

raw_by_poem = defaultdict(list)
with RAW_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        r = json.loads(line)
        raw_by_poem[r["poem_id"]].append(r)

### Inspect bayts under selected axis

Each ghazal assigned to the selected axis is printed in full to confirm
semantic alignment.


In [11]:
for r in matching:
    pid = r["poem_id"]
    print("=" * 60)
    print(f"poem_id = {pid}")
    print("AXIS:", r["ghazal_axis"])
    print("BAYTS:")
    for b in raw_by_poem[pid]:
        print("-", b["text"])

poem_id = 7
AXIS: جستجوی حقیقت پنهان
BAYTS:
- صوفی بیا که آینه، صافی‌ست جام را / تا بنگری صفای می لعل‌فام را
- راز درون پرده ز رندان مست پرس / کاین حال نیست زاهد عالی‌مقام را
- عنقا، شکار کس نشود، دام بازچین / که آنجا، همیشه، باد به دست است، دام را
- در بزم دور، یک‌دو قدح درکش و برو / یعنی طمع مدار وصال مدام را
- ای دل! شباب رفت و نچیدی گلی ز عیش / پیرانه‌سر مکن هنری ننگ و نام را
- در عیش نقد کوش که چون آبخور نماند / آدم بهشت، روضه‌ی دارالسلام را
- ما را بر آستان تو، بس حق خدمت است / ای خواجه! بازبین به ترحم غلام را
- «حافظ» مرید جام می است، ای صبا! برو! / وز بنده بندگی برسان شیخ جام را!
poem_id = 19
AXIS: جستجوی حقیقت پنهان
BAYTS:
- ای نسیم سحر آرامگه یار کجاست؟ / منزل آن مه عاشق‌کش عیار کجاست؟
- شب تار است و ره وادی ایمن در پیش / آتش طور کجا موعد دیدار کجاست؟
- هر که آمد به جهان نقش خرابی دارد / در خرابات بگویید که هشیار کجاست؟
- آن‌کس است اهل بشارت که اشارت داند / نکته‌ها هست بسی محرم اسرار کجاست؟
- هر سر موی مرا با تو هزاران کار است / ما کجاییم و ملامت‌گر بی‌کار کجاست؟
- باز پرسید 

### Inventory of all semantic axes

This provides a full human-readable overview of the abstract semantic
space covered by the dataset.

In [12]:
for axis in sorted(set(axes)):
    print(axis)

آرامش جویندگی در ناپایداری دنیا
آرزوی بازگشت
آرزوی دوام سلامت
آرزوی وصال
آرمان رهایی از قیود هستی
آرمان وصال و هم‌نشینی
آرمان‌گرایی در پیوند انسانی
آزادی اندیشه و رهایی از قیود اجتماعی
آمادگی و شایستگی برای تحول
آگاهی از ارزش لحظه‌ها
آیین همنشینی و آداب حضور
آیین وفاداری و همدلی
ابهام حقیقت و ظاهر
ابهام در رفتار و جایگاه
ابهام سرنوشت
ادراک اسرار هستی
ادراک حقیقت پنهان
ارزش فرصت‌ها و پیوندهای انسانی
ارزشمندی وصال و همنشینی
ارزش‌های درونی و پیوندهای انسانی
ارزش‌های متعالی زندگی
ارزش‌های معنوی در برابر مظاهر دنیوی
ارزش‌گذاری امور انسانی در برابر ناپایداری جهان
ارزش‌گذاری معرفت
ارزش‌گذاری و سنجش حقیقت
ارزش‌یابی همراهی و گذر زمان
اصالت و خلوص در سلوک انسانی
اعتدال در داوری و رفتار
امکان جستجوی جایگزین‌ها
امکان رهایی و بازگشت
امکان و تحول
امکان و تصادف دست‌یابی
امکانات مغفول
امکان‌های نهفته در پیوندهای معنوی
امکان‌گشایی و رفع انسداد
امید بازگشت
امید به بازگشت مطلوب
امید به تحول احوال
امید به دگرگونی و رهایی
امید به دگرگونی وضعیت
امید وصال
امیدواری به تحول احوال
امیدواری به دگرگونی و گشایش
ام

### Random global sanity check

A small random sample of ghazals is inspected to ensure no anomalies
remain after all validation steps.

In [13]:
random.seed(42)

for r in random.sample(rows, 3):
    print("AXIS:", r["ghazal_axis"])
    print("BAYTS:")
    for b in raw_by_poem[r["poem_id"]]:
        print("-", b["text"])
    print("-" * 40)

AXIS: جستجوی کمال معنوی
BAYTS:
- من که باشم که بر آن خاطر عاطر گذرم؟ / لطف‌ها می‌کنی ای خاک درت، تاج سرم
- دلبرا بنده نوازیت که آموخت؟ بگو / که من این ظن، به رقیبان تو هرگز نبرم
- همتم بدرقه‌ی راه کن ای طایر قدس / که دراز است ره مقصد و من نوسفرم
- ای نسیم سحری بندگی من برسان / که فراموش مکن وقت دعای سحرم
- خرم آن روز کز این مرحله بربندم بار / و از سر کوی تو پرسند رفیقان خبرم
- حافظا شاید اگر در طلب گوهر وصل / دیده دریا کنم از اشک و در او غوطه خورم
- پایه‌ی نظم بلند است و جهان گیر بگو / تا کند پادشه بحر دهان پر گهرم
----------------------------------------
AXIS: تجلی اراده و زیبایی مطلق
BAYTS:
- سر ارادت ما و آستان حضرت دوست / که هر چه بر سر ما می‌رود ارادت اوست
- نظیر دوست ندیدم، اگر چه از مه و مهر / نهادم آینه‌ها در مقابل رخ دوست
- صبا ز حال دل تنگ ما چه شرح دهد؟ / که چون شکنج ورق‌های غنچه تو بر توست
- نه من سبوکش این دیر رندسوزم و بس / بسا سرا که در این کارخانه سنگ و سبوست
- مگر تو شانه زدی زلف عنبرافشان را؟ / که باد غالیه‌سا گشت و خاک عنبربوست
- نثار روی تو هر برگ گل که در چمن است /